# Рекомендация тарифов

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


## Откроем и изучим данные

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(df.head(10));
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**ВЫВОД:** Мы будем иметь дело с 3214 клиентами. Судя по данным датасета, все необходимые признаки для определения подписки присутствуют. Также, уже добавлен столбец с видом подписки, который является целью классификации, что облегчает нам задачу. 

## Разбиение данных на выборки

Нужно разбить данные на 3 выборки
- обучающая выборка
- валидационная выборка
- тестовая выборка

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

df_train, df_valid_and_test = train_test_split(df, test_size=0.6, random_state=12345)
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.50, random_state=12345)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

print(features.shape)
print(target.shape)
print()
print('Размеры обучающей выборки:')
print(features_train.shape)
print(target_train.shape)
print()
print('Размеры валидационной выборки:')
print(features_valid.shape)
print(target_valid.shape)
print()
print('Размеры тестовой выборки:')
print(features_test.shape)
print(target_test.shape)

(3214, 4)
(3214,)

Размеры обучающей выборки:
(1285, 4)
(1285,)

Размеры валидационной выборки:
(964, 4)
(964,)

Размеры тестовой выборки:
(965, 4)
(965,)


**ВЫВОД**: Мы получили 3 выборки в пропорциях 60/20/20. Обучающая выборка забрала 60% данных, тестовая и валидационная поделили оставшиеся 40%.

## Исследование моделей

Первой рассмотрим модель дерева решений. Воспользуемся модулем GridSearchCV для поиска лучших гиперпараметров.

In [5]:
model = DecisionTreeClassifier()
parametrs = { 'max_depth': range (1, 11),
             'min_samples_leaf': range (1, 11),
             'min_samples_split': range (2, 10) }

grid = GridSearchCV(model, parametrs);
grid.fit(features_train, target_train);

In [6]:
grid.best_params_

{'max_depth': 8, 'min_samples_leaf': 10, 'min_samples_split': 6}

Укажем полученные значения в параметрах классификатора `DecisionTreeClassifier`.

In [7]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_leaf=10, min_samples_split=4 )
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print(f"Accuracy лучшей модели: {result}")

Accuracy лучшей модели: 0.8101659751037344


Точность предсказания с помощью дерева решений составляет 0.81

Далее рассмотрим модель случайного леса.

In [8]:
best_model = None
best_result = 0
for est in range(1, 51):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print(f"Accuracy наилучшей модели на валидационной выборке: {best_result}, Количество деревьев: {best_est}, Максимальная глубина: {depth}")

Accuracy наилучшей модели на валидационной выборке: 0.8215767634854771, Количество деревьев: 34, Максимальная глубина: 10


При `n_estimators = 34` и `max_depth = 10`, точность предсказания составляет 0.82

Теперь снова обратимся к модулю GridSearchCV для поиска лучших гиперпараметров.

In [9]:
model = RandomForestClassifier()
parametrs = { 'max_depth': range (1, 13, 2),
             'n_estimators': range (10, 51, 10),
             'min_samples_leaf': range (1, 11),
             'min_samples_split': range (2, 10, 2) }
grid = GridSearchCV(model, parametrs);
grid.fit(features_train, target_train);

In [10]:
grid.best_params_

{'max_depth': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 50}

In [11]:
model = RandomForestClassifier(random_state=12345, max_depth=9, min_samples_leaf=3, min_samples_split=4, n_estimators=20 )
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print(f"Accuracy лучшей модели: {result}")

Accuracy лучшей модели: 0.8132780082987552


И, проверим модель логистической регрессии.

In [12]:
best_result = 0
best_iter = 0
for iters in range(100, 1500):
    model = LogisticRegression(random_state=12345, solver='liblinear', max_iter = iters) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:        
        best_result = result   
        best_iter = iters
print(f"Accuracy лучшей модели: {best_result} при количестве итераций {best_iter}")        

Accuracy лучшей модели: 0.766597510373444 при количестве итераций 100


In [13]:
best_result = 0
for iters in range(100, 1500):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter = iters) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:        
        best_result = result   
print(f"Accuracy лучшей модели: {best_result} при количестве итераций {best_iter}") 

Accuracy лучшей модели: 0.7686721991701245 при количестве итераций 100


**ВЫВОД:** Как мы видим, самая высокая доля правильных ответов у модели случайного леса. Её и будем использовать для проверки на тестовой выборке.

## Проверка модели на тестовой выборке

In [14]:
model = RandomForestClassifier(random_state=12345, max_depth=9, min_samples_leaf=3, min_samples_split=4, n_estimators= 34)
model.fit(features_train, target_train)

predictions_train = model.predict(features)
predictions_test = model.predict(features_test)

def error_count(answers, predictions):
    errors = 0
    for i, k in zip(answers, predictions):
        if i != k:
            errors += 1
    return errors  
print(f"Ошибок в тестовой выборке относительно общего объема данных выборки: {error_count(target_test, predictions_test) / len(target_test):.1%}")
print(f"Ошибок в обучающей выборке относительно общего объема данных выборки: {error_count(target, predictions_train)/ len(target):.1%}")

Ошибок в тестовой выборке относительно общего объема данных выборки: 20.8%
Ошибок в обучающей выборке относительно общего объема данных выборки: 17.2%


In [15]:
accuracy1 = accuracy_score(target, predictions_train)
accuracy2 = accuracy_score(target_test, predictions_test)
print("Accuracy")
print("Обучающая выборка:", accuracy1)
print("Тестовая выборка:", accuracy2)

Accuracy
Обучающая выборка: 0.8276291225886746
Тестовая выборка: 0.7917098445595855


**ОБЩИЙ ВЫВОД**: Нам удалось достичь `Accuracy` выше 0.75. Наша модель может предсказывать правильные ответы с точностью до 79%.

## Проверка модели на адекватность

In [34]:
display(df['is_ultra'].loc[df['is_ultra'] == 1].count() / len(df))
display(df['is_ultra'].loc[df['is_ultra'] == 0].count() / len(df))

0.30647168637212197

0.693528313627878

**Вывод**: Ни одна из моделей не выдает качество ниже 70%. Соответственно, модели нельзя считать неадекватными.